Ceci est notre projet.

In [6]:
import numpy as np
import numpy.random as rnd
import itertools

Initialize the classes

In [4]:


class Bird:
    """A configuration of bird parameters"""

    def __init__(self, X, Y, theta, V):
        self.X = X
        self.Y = Y
        self.theta = theta
        self.velocity = V
    




    def get_neighbors(self, swarm, R):
        "get the neighbors of a bird"

        #verify that swarm is a class and print error if not
        if not isinstance(swarm, Swarm):
            raise TypeError("swarm must be a Swarm class")

        neighbors = []
        for bird in swarm:
            if bird != self:
                if np.sqrt((self.X - bird.X)**2 + (self.Y - bird.Y)**2) <= R:
                    neighbors.append(bird)
        return neighbors
    
    def get_mean_theta(self, neighbors):
        "get the average theta of the neighbors, according to the formula in the paper"

        thetas = [bird.theta for bird in neighbors]
        mean = np.mean(np.sin(thetas))/np.mean(np.cos(thetas))
        return np.arctan(mean)

    # def evolve(self, dt):
    #     "get the new position of the bird"

    #     self.X += self.velocity * np.cos(self.theta) * dt
    #     self.Y += self.velocity * np.sin(self.theta) * dt
    #     self.theta += self.get_mean_theta(self.get_neighbors(swarm, R)) * dt
    
class Swarm :
    "Creats the swarm state with many birds"

    def __init__(self, L, rho, V, eta, radius):
        self.length = L
        self.rho = rho
        self.velocity_norm = V
        self.eta = eta
        self.interaction_radius = radius

        self.dt = 1
        self.number = int(rho * (L**2))
        self.birds = []

    def initialize(self):
        "initialize the swarm with random positions and velocities"

        for i in range(self.number):
            X = rnd.uniform(0, self.length)
            Y = rnd.uniform(0, self.length)
            theta = rnd.uniform(0, 2*np.pi)
            self.birds.append(Bird(X, Y, theta, self.velocity_norm))

    def get_swarm_mean_velocity(self):
        "get the mean vectorial velocity of the swarm, which is the order parameter"

        mean_vx = np.mean([bird.velocity * np.cos(bird.theta) for bird in self.birds])
        mean_vy = np.mean([bird.velocity * np.sin(bird.theta) for bird in self.birds])

        return np.sqrt(mean_vx**2 + mean_vy**2)/self.velocity_norm

    def evolve(self):
        "evolve the swarm"
        
        update = []

        for bird in self.birds:
            new_X = bird.X + bird.velocity * np.cos(bird.theta) * self.dt    # birds are indistinguishable, so it doesn't matter which way the list goes
            new_Y = bird.Y + bird.velocity * np.sin(bird.theta) * self.dt
            random_theta = np.random.uniform(-self.eta/2, self.eta/2)
            new_theta = random_theta + bird.get_mean_theta(bird.get_neighbors(swarm, interaction_radius))

            update.append([new_X, new_Y, new_theta])
        
        #enumerate self.birds to apply the update
        for i, bird in enumerate(self.birds):
            bird.X = update[i][0]
            bird.Y = update[i][1]
            bird.theta = update[i][2]



Do the animation

In [7]:
#create swarm of birds and do an animation of the evolution
L = 10
rho = 0.5
V = 1
eta = 0.5
interaction_radius = 1
swarm = Swarm(L, rho, V, eta, interaction_radius)
swarm.initialize()